In [1]:
import logging
import requests
import pandas as pd
import sqlite3

In [2]:
log_format = "%(asctime)s::%(levelname)s::%(name)s::""%(filename)s::%(lineno)d::%(message)s"
logging.basicConfig(filename='./logs/get_coin_list.log', level='DEBUG', format=log_format)

In [3]:
def get_coin_list():
    """Gets a list of coins from the coingecko api, and returns a dataframe with a list of coins, requieres pandas as pd and requests"""
    #We could save the data as a json with
    #list_of_coins.to_json('coin_list.json', orient='records',lines=True)

    coins_url = 'https://api.coingecko.com/api/v3/coins/list'

    try:
        request = requests.get(coins_url)
        coins = request.json()
        list_of_coins = pd.json_normalize(coins)
    except requests.exceptions.RequestException as e:
        logging.info(f"Exception - {e} occured")
        return
    finally:
        status = None
        if request.status_code != 200:
            status = "FAIL"
            logging.info(f"List of coins : {status}")
        else:
            status = "PASS"
            logging.info(f"List of coins : {status}")
    
    list_of_coins.to_csv('./data/list_of_coins/list_of_coins.csv', index=False)

    return list_of_coins

In [4]:
get_coin_list.__doc__

'Gets a list of coins from the coingecko api, and returns a dataframe with a list of coins, requieres pandas as pd and requests'

In [5]:
coins = get_coin_list()

In [6]:
coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0vix-protocol,vix,0VIX Protocol
3,0x,zrx,0x Protocol
4,0x0-ai-ai-smart-contract,0x0,0x0.ai: AI Smart Contract
...,...,...,...
9986,zynergy,zyn,Zynergy
9987,zyro,zyro,Zyro
9988,zyrri,zyr,Zyrri
9989,zyx,zyx,ZYX


In [7]:
bitcoin_id = coins.loc[coins['name']=='Bitcoin','id'].values[0]

In [8]:
bitcoin_id

'bitcoin'

In [9]:
conn = sqlite3.connect('./db/coingecko')
c = conn.cursor()

In [10]:
c.execute('CREATE TABLE IF NOT EXISTS coin_list (id, symbol, name)')
conn.commit()

In [11]:
coins.to_sql('coin_list', conn, if_exists='replace')

9991